### Importing Dataset and LIbrary

In [12]:
pip install dataframe-image

Note: you may need to restart the kernel to use updated packages.


In [6]:
import sqlite3
import pandas as pd
import dataframe_image as dfi

### Connect Database

In [2]:
conn = sqlite3.connect("C:/Users/ASUS/Downloads/Porto/Bikestore/Bikestore.db")
cursor = conn.cursor()

In [3]:
query = 'select name from sqlite_master where type = "table" and name not like "sqlite_%"'
cursor.execute(query)
cursor.fetchall()

[('brands',),
 ('categories',),
 ('customers',),
 ('order_items',),
 ('orders',),
 ('products',),
 ('staffs',),
 ('stocks',),
 ('stores',)]

In [4]:
query = """
SELECT * FROM customers
"""
pd.read_sql(query, conn)


,customer_id,first_name,last_name,phone,email,street,city,state,zip_code
0,1,Debra,Burks,NULL,debra.burks@yahoo.com,9273 Thorne Ave.,Orchard Park,NY,14127
1,2,Kasha,Todd,NULL,kasha.todd@yahoo.com,910 Vine Street,Campbell,CA,95008
2,3,Tameka,Fisher,NULL,tameka.fisher@aol.com,769C Honey Creek St.,Redondo Beach,CA,90278
3,4,Daryl,Spence,NULL,daryl.spence@aol.com,988 Pearl Lane,Uniondale,NY,11553
4,5,Charolette,Rice,(916) 381-6003,charolette.rice@msn.com,107 River Dr.,Sacramento,CA,95820
...,...,...,...,...,...,...,...,...,...
1440,1441,Jamaal,Morrison,NULL,jamaal.morrison@msn.com,796 SE. Nut Swamp St.,Staten Island,NY,10301
1441,1442,Cassie,Cline,NULL,cassie.cline@gmail.com,947 Lafayette Drive,Brooklyn,NY,11201
1442,1443,Lezlie,Lamb,NULL,lezlie.lamb@gmail.com,401 Brandywine Street,Central Islip,NY,11722
1443,1444,Ivette,Estes,NULL,ivette.estes@gmail.com,88 N. Canterbury Ave.,Canandaigua,NY,14424


## Answering Questions 

##### 1. Retrieve the names and email addresses of all customers.

In [180]:
query = """
SELECT first_name || ' ' || last_name AS full_name, email
FROM customers
"""
df = pd.read_sql(query, conn)
df


,full_name,email
0,Debra Burks,debra.burks@yahoo.com
1,Kasha Todd,kasha.todd@yahoo.com
2,Tameka Fisher,tameka.fisher@aol.com
3,Daryl Spence,daryl.spence@aol.com
4,Charolette Rice,charolette.rice@msn.com
...,...,...
1440,Jamaal Morrison,jamaal.morrison@msn.com
1441,Cassie Cline,cassie.cline@gmail.com
1442,Lezlie Lamb,lezlie.lamb@gmail.com
1443,Ivette Estes,ivette.estes@gmail.com


In [ ]:
dfi.export(df,'no_1.png')

##### 2. List the product names and their respective categories.

In [181]:
query = """
SELECT p.product_name, c.category_name
FROM products p
JOIN categories c
ON p.category_id=c.category_id
"""
df = pd.read_sql(query,conn)
df

,product_name,category_name
0,Trek 820 - 2016,Mountain Bikes
1,Ritchey Timberwolf Frameset - 2016,Mountain Bikes
2,Surly Wednesday Frameset - 2016,Mountain Bikes
3,Trek Fuel EX 8 29 - 2016,Mountain Bikes
4,Heller Shagamaw Frame - 2016,Mountain Bikes
...,...,...
316,Trek Checkpoint ALR 5 - 2019,Road Bikes
317,Trek Checkpoint ALR 5 Women's - 2019,Road Bikes
318,Trek Checkpoint SL 5 Women's - 2019,Road Bikes
319,Trek Checkpoint SL 6 - 2019,Road Bikes


In [20]:
dfi.export(df,'no_2a.png')

In [182]:
#This code count how many bikes in each category
query = """
SELECT COUNT(p.product_name) AS "Total Product", c.category_name
FROM products p
JOIN categories c
ON p.category_id=c.category_id
GROUP BY 2
ORDER BY 1 DESC
"""
df = pd.read_sql(query,conn)
df

,Total Product,category_name
0,78,Cruisers Bicycles
1,60,Road Bikes
2,60,Mountain Bikes
3,59,Children Bicycles
4,30,Comfort Bicycles
5,24,Electric Bikes
6,10,Cyclocross Bicycles


In [22]:
dfi.export(df,'no_2b.png')

##### 3. Show the total number of orders placed.

In [183]:
query = """
SELECT COUNT(order_id) AS "Total Order", order_status
FROM orders
GROUP BY 2
"""
df = pd.read_sql(query,conn)
df

,Total Order,order_status
0,62,1
1,63,2
2,45,3
3,1445,4


In [24]:
dfi.export(df,'no_3.png')

In [184]:
query = """
WITH Order_Counts AS (
SELECT COUNT(*) OVER () AS Total_Orders, order_status,
COUNT(order_id) OVER (PARTITION BY order_status) AS Order_status
FROM orders
)
SELECT  
order_status, Total_Orders, Order_status
FROM Order_Counts
"""
pd.read_sql(query,conn)

,order_status,Total_Orders,order_status
0,1,1615,1
1,1,1615,1
2,1,1615,1
3,1,1615,1
4,1,1615,1
...,...,...,...
1610,4,1615,4
1611,4,1615,4
1612,4,1615,4
1613,4,1615,4


##### 4. Display the order ID, order date, and total amount for each order.

In [185]:
query = """
SELECT order_date, COUNT(order_id) AS Total_Order 
FROM orders
GROUP BY 1
ORDER BY 2 DESC

"""
df = pd.read_sql(query,conn)
df

,order_date,Total_Order
0,2018-04-17,9
1,2018-04-12,9
2,2017-10-01,8
3,2017-08-19,8
4,2018-04-06,7
...,...,...
720,2016-01-23,1
721,2016-01-15,1
722,2016-01-08,1
723,2016-01-06,1


In [142]:
dfi.export(df,'no_4.png')

##### 5. Find the customers who have spent more than $500 in total.

In [33]:
query = """
WITH Count_Spend AS (
SELECT order_id
FROM order_items
),

Customer_list AS (
SELECT customer_id, first_name, last_name
FROM customers
)

SELECT c.first_name, cs.order_id
FROM orders
JOIN Count_Spend cs ON cs.order_id = orders.order_id
JOIN Customer_list c ON c.customer_id = orders.customer_id

"""

pd.read_sql(query, conn)

,first_name,order_id
0,Johnathan,1
1,Johnathan,1
2,Johnathan,1
3,Johnathan,1
4,Johnathan,1
...,...,...
4717,Dorthey,1614
4718,Dorthey,1614
4719,Sarita,1615
4720,Sarita,1615


In [13]:
query = """
SELECT 
        c.customer_id, 
        c.first_name, 
        c.last_name,
        o.order_id
    FROM 
        customers c
    JOIN orders o ON o.customer_id = c.customer_id
"""

pd.read_sql(query,conn)

,customer_id,first_name,last_name,order_id
0,1,Debra,Burks,599
1,1,Debra,Burks,1555
2,1,Debra,Burks,1613
3,2,Kasha,Todd,692
4,2,Kasha,Todd,1084
...,...,...,...,...
1610,1441,Jamaal,Morrison,1366
1611,1442,Cassie,Cline,1036
1612,1443,Lezlie,Lamb,558
1613,1444,Ivette,Estes,616


In [17]:
query = """
SELECT 
        o.order_id,
        SUM(oi.quantity * oi.list_price * (1 - oi.discount)) AS Spending
FROM order_items oi
JOIN orders o ON o.order_id = oi.order_id
GROUP BY 1
"""

pd.read_sql(query,conn)

,order_id,Spending
0,1,10231.0464
1,2,1697.9717
2,3,1519.9810
3,4,1349.9820
4,5,3900.0607
...,...,...
1610,1611,8963.9647
1611,1612,3781.1284
1612,1613,5257.9717
1613,1614,6104.0354


In [146]:
query = """
WITH Count_Spend AS (
 SELECT 
        o.order_id,
        SUM(oi.quantity * oi.list_price * (1 - oi.discount)) AS Spending
FROM order_items oi
JOIN orders o ON o.order_id = oi.order_id
GROUP BY 1
),
Customers_list AS (
    SELECT 
        c.customer_id, 
        c.first_name, 
        c.last_name,
        o.order_id
    FROM 
        customers c
    JOIN orders o ON o.customer_id = c.customer_id
)
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name AS full_name, 
    SUM(cs.Spending) AS "TOTAL SPENDING"
FROM 
    Count_Spend cs
JOIN 
    Customers_list c ON c.order_id = cs.order_id
GROUP BY 1
HAVING "TOTAL SPENDING" > 30000
ORDER BY 3 DESC
"""

df = pd.read_sql(query, conn)
df

,customer_id,full_name,TOTAL SPENDING
0,94,Sharyn Hopkins,34807.9392
1,10,Pamelia Newman,33634.2604
2,75,Abby Gamble,32803.0062
3,6,Lyndsey Bean,32675.0725
4,16,Emmitt Sanchez,31925.8857
5,73,Melanie Hayes,31913.6902


In [147]:
dfi.export(df,'no_5.png')

In [26]:
query = """
SELECT c.customer_id,
SUM(oi.quantity*(oi.list_price*(1-oi.discount))) AS Spending
FROM orders o
JOIN customers c ON c.customer_id = o.customer_id
JOIN order_items oi ON oi.order_id = o.order_id
GROUP BY 1
HAVING Spending > 500
ORDER BY 2 DESC
"""

pd.read_sql(query, conn)

,customer_id,Spending
0,94,34807.9392
1,10,33634.2604
2,75,32803.0062
3,6,32675.0725
4,16,31925.8857
...,...,...
1370,1130,502.1814
1371,964,502.1814
1372,613,502.1814
1373,608,502.1814


##### 6. List the top 3 best-selling products in terms of quantity.

In [150]:
query = """
SELECT oi.product_id, p.product_name, oi.quantity, 
SUM(oi.quantity*(oi.list_price*(1-oi.discount))) AS Sales
FROM order_items oi
JOIN products p ON p.product_id = oi.product_id
GROUP BY 1
ORDER BY 3 DESC
LIMIT 3
"""

df = pd.read_sql(query, conn)
df

,product_id,product_name,quantity,Sales
0,315,Electra Townie Original 21D Ladies' - 2018,2,1063.981
1,314,Electra Townie Original 21D EQ Ladies' - 2018,2,1087.984
2,313,Electra Townie Original 1 Ladies' - 2018,2,1574.965


In [149]:
dfi.export(df,'no_6.png')

#### 7.  Show the average order amount for each month in 2016-2018.

In [186]:
query = """
WITH Count_Date AS (
SELECT COUNT(order_id) AS Total, STRFTIME("%Y-%m-%d", order_date) AS Date
FROM orders
GROUP BY 2
)
SELECT ROUND(AVG(Total)) AS AVG_Order, STRFTIME("%Y-%m", Date) AS Month
FROM Count_Date
WHERE Month LIKE '%2018%' OR Month LIKE '%2017%' OR Month LIKE '%2016%'
GROUP BY 2
ORDER BY 1 DESC

"""

df = pd.read_sql(query, conn)
df

,AVG_Order,Month
0,4.0,2018-04
1,3.0,2018-03
2,3.0,2017-10
3,3.0,2017-06
4,3.0,2017-03
5,2.0,2018-02
6,2.0,2018-01
7,2.0,2017-12
8,2.0,2017-11
9,2.0,2017-09


In [157]:
dfi.export(df,'no_7.png')

#### 8. Identify the customer who placed the most orders.

In [187]:
query = """
SELECT o.customer_id, 
c.first_name || ' ' || c.last_name AS full_name,
COUNT(o.customer_id) AS Total_Order
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
GROUP BY 1
ORDER BY 3 DESC

"""

df = pd.read_sql(query, conn)
df

,customer_id,full_name,Total_Order
0,116,Merlene Vinson,3
1,77,Keri Bridges,3
2,68,Jayne Kirkland,3
3,66,Lorrie Becker,3
4,64,Bobbie Foster,3
...,...,...,...
1440,100,Kellie Franco,1
1441,99,Carie Kidd,1
1442,98,Violet Valenzuela,1
1443,96,Floretta Higgins,1


In [55]:
dfi.export(df,'no_8.png')

#### 9. Display the products that have not been sold yet.

In [159]:
q = """
SELECT p.product_name, o.order_id
FROM products p
LEFT JOIN order_items o ON p.product_id = o.product_id
WHERE o.order_id IS NULL
"""
df = pd.read_sql(q, conn)
df

,product_name,order_id
0,Trek 820 - 2016,None
1,Surly Krampus Frameset - 2018,None
2,Trek Kids' Dual Sport - 2018,None
3,Trek Domane SLR 6 Disc Women's - 2018,None
4,Electra Townie Go! 8i Ladies' - 2018,None
5,Trek Precaliber 12 Girl's - 2018,None
6,Electra Savannah 1 (20-inch) - Girl's - 2018,None
7,Electra Sweet Ride 1 (20-inch) - Girl's - 2018,None
8,Trek Checkpoint ALR 4 Women's - 2019,None
9,Trek Checkpoint ALR 5 - 2019,None


In [57]:
dfi.export(df,'no_9.png')

#### 10. Calculate the total revenue generated from each product category.

In [58]:
query = """
WITH Category_Product AS (
SELECT p.category_id, c.category_name, p.product_id
FROM products p
JOIN categories c ON p.category_id = c.category_id
)
SELECT 
CAST(SUM((oi.quantity*oi.list_price)*(1-oi.discount)) AS INT) AS Revenue,
cp.category_name
FROM order_items oi
JOIN Category_Product cp ON oi.product_id = cp.product_id
GROUP BY 2
ORDER BY 1 DESC
"""
df = pd.read_sql(query,conn)
df

,Revenue,category_name
0,2715079,Mountain Bikes
1,1665098,Road Bikes
2,995032,Cruisers Bicycles
3,916684,Electric Bikes
4,711011,Cyclocross Bicycles
5,394020,Comfort Bicycles
6,292189,Children Bicycles


In [59]:
dfi.export(df,'no_10.png')

## Study Case

### 1. Revenue Analysis:

- Calculate the total revenue generated by each product category.
- Determine the top 5 products with the highest revenue.
- Identify the top 3 customers who have spent the most on purchases.

In [60]:
#Calculate the total revenue generated by each product category
query = """
WITH Category_Product AS (
SELECT p.category_id, c.category_name, p.product_id, p.product_name
FROM products p
JOIN categories c ON p.category_id = c.category_id
)
SELECT 
CAST(SUM((oi.quantity*oi.list_price)*(1-oi.discount)) AS INT) AS Revenue,
cp.category_name
FROM order_items oi
JOIN Category_Product cp ON oi.product_id = cp.product_id
GROUP BY 2
ORDER BY 1 DESC
"""
df = pd.read_sql(query,conn)
df

,Revenue,category_name
0,2715079,Mountain Bikes
1,1665098,Road Bikes
2,995032,Cruisers Bicycles
3,916684,Electric Bikes
4,711011,Cyclocross Bicycles
5,394020,Comfort Bicycles
6,292189,Children Bicycles


In [61]:
dfi.export(df,'rev_1.png')

In [188]:
#Determine the top 5 products with the highest revenue.
query = """
WITH Category_Product AS (
SELECT p.category_id, c.category_name, p.product_id, p.product_name
FROM products p
JOIN categories c ON p.category_id = c.category_id
)
SELECT 
cp.product_name,
CAST(SUM((oi.quantity*oi.list_price)*(1-oi.discount)) AS INT) AS Revenue
FROM order_items oi
JOIN Category_Product cp ON oi.product_id = cp.product_id
GROUP BY 1
ORDER BY 2 DESC

"""
df = pd.read_sql(query,conn)
df

,product_name,Revenue
0,Trek Slash 8 27.5 - 2016,555558
1,Trek Conduit+ - 2016,389248
2,Trek Fuel EX 8 29 - 2016,368472
3,Surly Straggler 650b - 2016,226765
4,Trek Domane SLR 6 Disc - 2017,211584
...,...,...
273,Trek Precaliber 24 7-speed Girl's - 2018,255
274,Electra Cruiser 1 Tall - 2016/2018,242
275,Trek MT 201 - 2018,224
276,Electra Soft Serve 1 (16-inch) - Girl's - 2018,223


In [63]:
dfi.export(df,'rev_2.png')

In [189]:
#Identify the top 3 customers who have spent the most on purchases
query = """
WITH Category_Product AS (
SELECT p.category_id, c.category_name, p.product_id, p.product_name
FROM products p
JOIN categories c ON p.category_id = c.category_id
),
Customer_list AS (
SELECT o.order_id, o.customer_id, 
c.first_name || ' ' || c.last_name AS full_name
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
)
SELECT 
cl.full_name,
SUM(oi.quantity) AS Quantity,
CAST(SUM((oi.quantity*oi.list_price)*(1-oi.discount)) AS INT) AS Spending,
COUNT(cl.order_id) AS Total_Order
FROM order_items oi
JOIN Category_Product cp ON oi.product_id = cp.product_id
JOIN Customer_list cl ON oi.order_id = cl.order_id
GROUP BY 1
ORDER BY 3 DESC

"""
df = pd.read_sql(query,conn)
df

,full_name,Quantity,Spending,Total_Order
0,Sharyn Hopkins,15,34807,10
1,Pamelia Newman,18,33634,11
2,Abby Gamble,11,32803,7
3,Lyndsey Bean,16,32675,11
4,Emmitt Sanchez,19,31925,12
...,...,...,...,...
1439,Lloyd Miranda,1,170,1
1440,Caroline Jenkins,1,170,1
1441,Carissa Foreman,1,170,1
1442,Luciana Mcgee,1,167,1


In [169]:
dfi.export(df,'rev_3.png')

### 2. Inventory Management:

- List products with low stock (quantity below a certain threshold).
- Determine the average time between order placement and shipment for each product.
- Identify products with a high rate of return (based on order history).

In [62]:
q = "SELECT * FROM stocks"

pd.read_sql(q, conn)

,store_id,product_id,quantity
0,1,1,27
1,1,2,5
2,1,3,6
3,1,4,23
4,1,5,22
...,...,...,...
934,3,309,30
935,3,310,8
936,3,311,23
937,3,312,18


In [66]:
#List products with low stock (quantity below a certain threshold).
#Counting the stocks from each stores
query = """
WITH product_stocks AS (
SELECT p.product_id, p.product_name, s.quantity, s.store_id
FROM products p
JOIN stocks s ON p.product_id = s.product_id
)
SELECT s.store_id, 
s.store_name,  
SUM(ps.quantity) AS Quantity
FROM stores s
JOIN product_stocks ps ON s.store_id = ps.store_id
GROUP BY 1
ORDER BY 3
"""
df = pd.read_sql(query,conn)
df

,store_id,store_name,Quantity
0,2,Baldwin Bikes,4359
1,1,Santa Cruz Bikes,4532
2,3,Rowlett Bikes,4620


In [173]:
#List products with low stock (quantity below a certain threshold).
#Counting the stocks based from the product in each stores
query = """
WITH product_stocks AS (
SELECT p.product_id, p.product_name, s.quantity, s.store_id
FROM products p
JOIN stocks s ON p.product_id = s.product_id
)
SELECT s.store_id, 
s.store_name, 
ps.product_id, 
ps.product_name, 
SUM(ps.quantity) AS Quantity
FROM stores s
JOIN product_stocks ps ON s.store_id = ps.store_id
WHERE ps.quantity = 0
GROUP BY 1,3
ORDER BY 5
"""
df = pd.read_sql(query,conn)
df

,store_id,store_name,product_id,product_name,Quantity
0,1,Santa Cruz Bikes,6,Surly Ice Cream Truck Frameset - 2016,0
1,1,Santa Cruz Bikes,8,Trek Remedy 29 Carbon Frameset - 2016,0
2,1,Santa Cruz Bikes,32,Trek Farley Alloy Frameset - 2017,0
3,1,Santa Cruz Bikes,42,Trek Fuel EX 5 27.5 Plus - 2017,0
4,1,Santa Cruz Bikes,92,Haro Shredder 20 - 2017,0
5,1,Santa Cruz Bikes,160,Trek Emonda SLR 6 - 2018,0
6,1,Santa Cruz Bikes,163,Surly Pack Rat - 2018,0
7,1,Santa Cruz Bikes,168,Surly Straggler - 2018,0
8,1,Santa Cruz Bikes,246,Electra Townie Original 3i EQ Ladies' - 2018,0
9,1,Santa Cruz Bikes,302,Electra Townie Original 1 - 2018,0


In [174]:
dfi.export(df,'inv_1.png')

In [190]:
#List products with low stock (quantity below a certain threshold).
#Counting the stocks based from the product only
query = """
SELECT p.product_id, p.product_name, SUM(s.quantity) AS Quantity
FROM products p
JOIN stocks s ON p.product_id = s.product_id
GROUP BY 1
ORDER BY 3
"""
pd.read_sql(query,conn)

,product_id,product_name,Quantity
0,175,Trek Domane SLR Frameset - 2018,5
1,289,"Electra Superbolt 1 20"" - 2018",9
2,220,Electra Cruiser 1 Ladies' - 2018,10
3,70,Electra Amsterdam Original 3i - 2015/2017,11
4,105,Sun Bicycles Streamway 7 - 2017,11
...,...,...,...
308,61,Trek Powerfly 8 FS Plus - 2017,78
309,109,Sun Bicycles Cruz 7 - Women's - 2017,79
310,196,Trek Verve+ - 2018,79
311,64,Electra Townie Original 7D - 2017,82


In [191]:
#Determine the average time between order placement and shipment for each product.
query = """
SELECT 
    oi.product_id,
    p.product_name,
    o.order_date AS Order_Date,
    o.shipped_date AS Shipped_Date,
    CAST(AVG(julianday(o.shipped_date) - julianday(o.order_date)) AS INTEGER) AS AVG_Day
FROM 
    orders o
JOIN 
    order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
GROUP BY 1
HAVING AVG_Day IS NOT NULL
ORDER BY 5 DESC
"""    
df = pd.read_sql(query, conn)
df

,product_id,product_name,Order_Date,Shipped_Date,AVG_Day
0,299,Electra Townie Original 21D - 2018,2018-03-19,2018-03-22,3
1,296,"Electra Treasure 3i 20"" - 2018",2018-01-06,2018-01-09,3
2,290,"Electra Superbolt 3i 20"" - 2018",2018-01-05,2018-01-08,3
3,287,Electra Straight 8 1 (16-inch) - Boy's - 2018,2018-03-28,2018-03-31,3
4,278,Trek Precaliber 24 21-speed Girl's - 2018,2018-03-26,2018-03-29,3
...,...,...,...,...,...
273,11,Surly Straggler 650b - 2016,2016-01-06,2016-01-09,1
274,10,Surly Straggler - 2016,2016-01-01,2016-01-03,1
275,9,Trek Conduit+ - 2016,2016-01-04,2016-01-05,1
276,7,Trek Slash 8 27.5 - 2016,2016-01-05,2016-01-08,1


In [178]:
dfi.export(df,'inv_2.png')

In [192]:
#Identify products with a high rate of return (based on order history).
query = """
WITH Return_Product AS (
SELECT 
    oi.product_id,
    p.product_name,
    COUNT(oi.order_id) OVER () AS Total_Return,
    COUNT(oi.order_id) AS Rate_Return

FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY 1
ORDER BY 3 DESC
)
SELECT 
    product_id,
    product_name,
    CASE 
        WHEN Total_Return > 0 THEN (Rate_Return * 100 / Total_Return)  -- Avoid division by zero
        ELSE 0  -- or NULL or any default value as per your requirement
    END AS 'Return_Rate (%)'
FROM Return_Product
ORDER BY 3 DESC

"""

df = pd.read_sql(query, conn)
df

,product_id,product_name,Return_Rate (%)
0,6,Surly Ice Cream Truck Frameset - 2016,35
1,12,Electra Townie Original 21D - 2016,33
2,13,Electra Cruiser 1 (24-Inch) - 2016,33
3,7,Trek Slash 8 27.5 - 2016,32
4,9,Trek Conduit+ - 2016,32
...,...,...,...
302,311,Electra Townie Commute 8D - 2018,0
303,312,Electra Townie Commute 8D Ladies' - 2018,0
304,313,Electra Townie Original 1 Ladies' - 2018,0
305,314,Electra Townie Original 21D EQ Ladies' - 2018,0


In [75]:
dfi.export(df,'inv_3.png')

### 3. Customer Segmentation:

- Segment customers based on their purchase history (e.g., frequent buyers, one-time purchasers, etc.).
    - Frequent Customers: These are customers who generate significant revenue for your business, either through large individual purchases or frequent purchases over time.
    - Infrequent Customers: These are customers who contribute smaller amounts to your revenue. They may make occasional purchases or purchase lower-priced items.
- Analyze the purchasing behavior of customers from different regions.

In [78]:
#Make query that describe the categorized for frequent buyers (buy more than 2) and 
#infrequent buyers (less than 3)
query = """
WITH customer_cat AS (
SELECT customer_id, 
COUNT(order_status) AS total_buyers
FROM orders
GROUP BY 1
ORDER BY 2 DESC
)
SELECT CASE 
    WHEN total_buyers > 2 THEN "Frequent Buyers"
    ELSE "Infrequent Buyers"
    END AS "Cust_Cat",
COUNT(customer_id) AS Total_Customers
FROM customer_cat
GROUP BY 1
"""
df = pd.read_sql(query, conn)
df

,Cust_Cat,Total_Customers
0,Frequent Buyers,39
1,Infrequent Buyers,1406


In [125]:
#Make query that describe the categorized for frequent buyers (buy more than 2) or spending > 5000 
#infrequent buyers (less than 3) or spending < 4999
query = """
WITH customer_cat AS (
    SELECT 
        order_id,
        customer_id, 
        COUNT(order_id) AS Total_Buy
    FROM 
        orders
    GROUP BY 
        customer_id
),
order_list AS (
    SELECT 
        o.customer_id, 
        oi.order_id, 
        oi.quantity,
        SUM(oi.quantity * oi.list_price * (1 - oi.discount)) AS Spending
    FROM 
        order_items oi
    JOIN 
        orders o ON o.order_id = oi.order_id
    GROUP BY 
        o.customer_id, oi.order_id
)
SELECT 
    cc.customer_id,
    CASE 
        WHEN cc.Total_Buy > 2 OR ol.Spending > 5000 THEN 'Frequent Buyers'
        ELSE 'Infrequent Buyers'
    END AS Cust_Cat,
    cc.Total_Buy,
    ol.Spending
FROM 
    customer_cat cc
JOIN 
    order_list ol ON cc.customer_id = ol.customer_id
GROUP BY 1
ORDER BY 2 ASC
"""
df = pd.read_sql(query, conn)
df

,customer_id,Cust_Cat,Total_Buy,Spending
0,1,Frequent Buyers,3,5257.9717
1,2,Frequent Buyers,3,1394.9814
2,3,Frequent Buyers,3,3781.1284
3,4,Frequent Buyers,3,1763.1910
4,5,Frequent Buyers,3,5098.1444
...,...,...,...,...
1440,1432,Infrequent Buyers,1,2696.9907
1441,1433,Infrequent Buyers,1,4226.2305
1442,1434,Infrequent Buyers,1,1530.9630
1443,1437,Infrequent Buyers,1,3799.9905


In [126]:
#counting customers categorized as Frequent Buyers and Infrequent Buyers
query = """
WITH customer_cat AS (
    SELECT 
        order_id,
        customer_id, 
        COUNT(order_id) AS Total_Buy
    FROM 
        orders
    GROUP BY 
        customer_id
),
order_list AS (
    SELECT 
        o.customer_id, 
        oi.order_id, 
        oi.quantity,
        SUM(oi.quantity * oi.list_price * (1 - oi.discount)) AS Sales
    FROM 
        order_items oi
    JOIN 
        orders o ON o.order_id = oi.order_id
    GROUP BY 
        o.customer_id, oi.order_id
)
SELECT 
    CASE 
        WHEN cc.Total_Buy > 2 OR ol.Sales > 5000 THEN 'Frequent Buyers'
        ELSE 'Infrequent Buyers'
    END AS Cust_Cat,
    COUNT(ol.customer_id) AS Total
FROM 
    customer_cat cc
JOIN 
    order_list ol ON cc.customer_id = ol.customer_id
GROUP BY 1
ORDER BY 2 ASC
"""
df = pd.read_sql(query, conn)
df

,Cust_Cat,Total
0,Frequent Buyers,666
1,Infrequent Buyers,949


In [127]:
dfi.export(df,'cs_1.png')

In [179]:
#I decide to make some segemntation that which region that buy this product, or maybe other region tend to buy this type of bike
# based on state and city only for top 5 customers
query = """
WITH customer_list AS (
SELECT COUNT(customer_id) AS Total_Cust, state, city, customer_id
FROM customers
GROUP BY 2, 3
HAVING COUNT(customer_id) > 10
ORDER BY 1 DESC
LIMIT 5
),
Order_list AS (
SELECT 
    o.order_id, oi.product_id, p.product_name, s.store_id, s.store_name, o.customer_id
FROM orders o
JOIN order_items oi ON oi.order_id = o.order_id
JOIN products p ON p.product_id = oi.product_id
JOIN stores s ON s.store_id = o.store_id
)
SELECT 
    cl.Total_Cust, cl.state, cl.city,
    (SELECT product_name FROM Order_list WHERE customer_id = cl.customer_id GROUP BY product_name ORDER BY COUNT(*) DESC LIMIT 1) AS Mode_Product,
    (SELECT store_name FROM Order_list WHERE customer_id = cl.customer_id GROUP BY store_name ORDER BY COUNT(*) DESC LIMIT 1) AS Mode_Store
FROM customer_list cl
JOIN Order_list ol ON cl.customer_id = ol.customer_id
GROUP BY cl.customer_id
ORDER BY 1 DESC
"""

df = pd.read_sql(query, conn)
df

,Total_Cust,state,city,Mode_Product,Mode_Store
0,20,NY,Mount Vernon,Ritchey Timberwolf Frameset - 2016,Baldwin Bikes
1,17,NY,Ballston Spa,Pure Cycles William 3-Speed - 2016,Baldwin Bikes
2,17,NY,Scarsdale,Trek Kids' Neko - 2018,Baldwin Bikes
3,14,NY,Canandaigua,Trek XM700+ - 2018,Baldwin Bikes
4,13,NY,Floral Park,Trek Silque SLR 8 Women's - 2017,Baldwin Bikes


In [133]:
dfi.export(df,'cs_2.png')